In [1]:
import os
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"
import torch
device = 0 if torch.cuda.is_available() else -1  # 0 = first GPU, -1 = CPU for speed
from arabert.preprocess import ArabertPreprocessor #preprocess text
from dotenv import load_dotenv
import re
from nltk.tokenize import sent_tokenize
import random
from transformers import pipeline


load_dotenv()  # Loads HF_TOKEN automatically

C:\Users\renad\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
#directly thro hugging face
from transformers import pipeline

text = "إمارة أبوظبي هي إحدى إمارات دولة الإمارات العربية المتحدة السبع"

# POS tagging
pos = pipeline(
    "token-classification",
    model="CAMeL-Lab/bert-base-arabic-camelbert-da-pos-msa",
    framework="pt",
    trust_remote_code=True  # VERY IMPORTANT
)

# NER
ner = pipeline(
    "ner",
    model="CAMeL-Lab/bert-base-arabic-camelbert-msa-ner",
    framework="pt",
    trust_remote_code=True  # VERY IMPORTANT
)

print("POS:", pos(text))
print("NER:", ner(text))


Some weights of the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-da-pos-msa were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu
Some weights of the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-msa-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoin

POS: [{'entity': 'noun', 'score': 0.9999914, 'index': 1, 'word': 'إمارة', 'start': 0, 'end': 5}, {'entity': 'noun_prop', 'score': 0.99924755, 'index': 2, 'word': 'أبوظبي', 'start': 6, 'end': 12}, {'entity': 'pron', 'score': 0.9999192, 'index': 3, 'word': 'هي', 'start': 13, 'end': 15}, {'entity': 'noun', 'score': 0.9999318, 'index': 4, 'word': 'إحدى', 'start': 16, 'end': 20}, {'entity': 'noun', 'score': 0.99999106, 'index': 5, 'word': 'إما', 'start': 21, 'end': 24}, {'entity': 'noun', 'score': 0.99998987, 'index': 6, 'word': '##رات', 'start': 24, 'end': 27}, {'entity': 'noun', 'score': 0.99999344, 'index': 7, 'word': 'دولة', 'start': 28, 'end': 32}, {'entity': 'noun', 'score': 0.99998987, 'index': 8, 'word': 'الإمارات', 'start': 33, 'end': 41}, {'entity': 'adj', 'score': 0.9999056, 'index': 9, 'word': 'العربية', 'start': 42, 'end': 49}, {'entity': 'adj', 'score': 0.9999796, 'index': 10, 'word': 'المتحدة', 'start': 50, 'end': 57}, {'entity': 'noun_num', 'score': 0.9993895, 'index': 11, '

In [5]:
def generate_question_text(entity, pos, word, sentence=None):
    """Generate natural Arabic question from entity + POS MQ."""
    
    # Person + adjective
    if entity == "PER" and pos == "adj":
        return f"من لديه {word}؟"
    
    # Person + proper noun
    if entity == "PER" and pos == "noun_prop":
        return f"من هو {word}؟"
    
    # Location + any noun
    if entity == "LOC":
        return f"أين يقع {word}؟"
    
    # Organization + adjective
    if entity == "ORG" and pos == "adj":
        return f"ما هي المنظمة التي وُصفت بأنها {word}؟"
    
    # Cloze/fill-in-the-blank
    if sentence:
        return f"أكمل الفراغ: {sentence.replace(word, '____')}"
    
    return f"صف الكلمة: {word}"


In [6]:
def generate_TF_question(sentence, entity_word):
    """
    Generate a True/False question by either keeping the entity (True)
    or replacing it with a distractor (False).
    """
    # True statement
    true_statement = f"{sentence} (صح أم خطأ؟)"
    
    # False statement: replace entity with simple fake word
    # In practice, replace with plausible distractors of same type
    distractor = "بديل"
    false_statement = sentence.replace(entity_word, distractor) + " (صح أم خطأ؟)"
    
    # Randomly choose True or False
    if random.choice([True, False]):
        return {"type": "TF", "statement": true_statement, "answer": True}
    else:
        return {"type": "TF", "statement": false_statement, "answer": False}


In [13]:
def generate_MCQ_question(entity_word, sentence, entity_type="LOC", top_k=3):
    """
    Generate MCQ using entity as correct answer and model-predicted distractors.
    """
    # 1️⃣ Prepare question text
    question_text = generate_question_text(entity_type, "noun", entity_word, sentence)
    
    # 2️⃣ Mask the entity in the sentence
    masked_sentence = sentence.replace(entity_word, "[MASK]")
    
    # 3️⃣ Get top predictions from the model
    predictions =ner(masked_sentence)
    
    # 4️⃣ Filter out the original word and take top_k as distractors
    distractors = [pred['token_str'] for pred in predictions if pred['token_str'] != entity_word][:top_k]
    
    # 5️⃣ Combine correct answer + distractors and shuffle
    options = [entity_word] + distractors
    random.shuffle(options)
    
    return {"type": "MCQ", "question": question_text, "options": options, "answer": entity_word}

In [10]:
def make_quiz(text):
    questions = []
    
    # Run models
    pos_tags = pos(text)
    ner_tags = ner(text)
    
    # Use NER to create TF + MCQ
    for ner_tag in ner_tags:
        word = ner_tag['word'] #the actual entity for example ahmad
        entity = ner_tag['entity'] #the entity label
        
        # Find POS of this word (simple match by string)
        pos_tag = next((p['entity'] for p in pos_tags if p['word'] == word), "noun")
        
        # Generate TF question
        questions.append(generate_TF_question(text, word))
        
        # Generate MCQ question
        questions.append(generate_MCQ_question(word, text, entity))
    
    return questions


In [11]:
text = "تأسست جامعة الملك سعود في الرياض عام 1957."
quiz = make_quiz(text)

for q in quiz:
    if q["type"] == "TF":
        print("TF:", q["statement"], "=>", q["answer"])
    elif q["type"] == "MCQ":
        print("MCQ:", q["question"])
        print("Options:", q["options"], "Answer:", q["answer"])


TF: تأسست جامعة الملك سعود في الرياض عام 1957. (صح أم خطأ؟) => True
MCQ: أكمل الفراغ: تأسست ____ الملك سعود في الرياض عام 1957.
Options: ['خيار1', 'جامعة', 'خيار3', 'خيار2'] Answer: جامعة
TF: تأسست جامعة الملك سعود في الرياض عام 1957. (صح أم خطأ؟) => True
MCQ: أكمل الفراغ: تأسست جامعة ____ سعود في الرياض عام 1957.
Options: ['الملك', 'خيار2', 'خيار1', 'خيار3'] Answer: الملك
TF: تأسست جامعة الملك سعود في الرياض عام 1957. (صح أم خطأ؟) => True
MCQ: أكمل الفراغ: تأسست جامعة الملك ____ في الرياض عام 1957.
Options: ['خيار3', 'سعود', 'خيار1', 'خيار2'] Answer: سعود
TF: تأسست جامعة الملك سعود في بديل عام 1957. (صح أم خطأ؟) => False
MCQ: أكمل الفراغ: تأسست جامعة الملك سعود في ____ عام 1957.
Options: ['خيار2', 'الرياض', 'خيار1', 'خيار3'] Answer: الرياض
